In [15]:
import networkx as nx
import numpy as np
import time

In [2]:
G=nx.DiGraph()
G.add_nodes_from([i for i in range(20)])
G.add_edges_from([(i,j) for i in range(20) for j in list(range(0,i,2))+list(range(i+1,20,2))])
Gr=G.reverse()

baseline:

* C-DFS

* BC-DFS

In [3]:
def C_DFS(G,s,t,k,ans,path=[]):
    INF=1e5
    if s==t:
        return 0
    if k<=0:
        return INF
    ret=INF
    path.append(s)
    for next in G[s]:
        if next not in path:
            tmp=C_DFS(G,next,t,k-1,ans,path)
            if tmp<INF-5:
                ans.append((s,next))
            ret=min(ret,tmp+1)
    path.pop()
    return ret

In [65]:
def BC_DFS(G,Gr,s,t,k,ans):
    def BC_DFS_recursive(G,Gr,s,t,k,ans,S,bar):
        def UpdateBar(Gr,u,b,bar):
            if bar[u]>b:
                bar[u]=b
                for prev in Gr[b]:
                    UpdateBar(Gr,prev,b+1,bar)

        F=k+1
        if s==t:
            return 0
        S.append(s)
        if len(S)<=k:
            for next in G[s]:
                if next not in S:
                    if len(S)+bar[next]<=k:
                        f=BC_DFS_recursive(G,Gr,next,t,k,ans,S,bar)
                        if f<k+1:
                            ans.append((s,next))
                            F=min(f+1,F)
        if F==k+1:
            bar[s]=k+1-(len(S)-1)
        else:
            UpdateBar(Gr,s,F,bar)
        S.pop()
        return F

    S=[]
    bar=[0 for i in range(len(G))]
    return BC_DFS_recursive(G,Gr,s,t,k,ans,S,bar)

bisearch

In [23]:
def backProp(Gr,ls,lt,k,v,ans):
    for prev in Gr[v]:
        if prev in Gr:
            if lt[prev]>lt[v]+1:
                if lt[prev]+ls[prev]>k and ls[prev]+lt[v]+1<=k:
                    ans.append(prev)
                lt[prev]=lt[v]+1
                if lt[prev]+ls[prev]<=k:
                    backProp(Gr,ls,lt,k,prev,ans)

def bisearch(G,Gr,s,t,k,ans):
    ls=[k+1 for i in range(len(G))]
    lt=[k+1 for i in range(len(G))]
    ls[s]=0
    lt[t]=0
    qs=[s]
    qt=[t]
    subGs=[s]
    subGt=[t]

    for i in range(k):
        if len(qs)<len(qt) and len(qs)>0:
            tqs=[]
            for i in qs:
                for next in G[i]:
                    if ls[next]>ls[i]+1:
                        tqs.append(next)
                        subGs.append(next)
                        if (ls[next]+lt[next])>k and (ls[i]+1+lt[next])<=k:
                            ans.append(next)
                        ls[next]=ls[i]+1
                        if(lt[next]+ls[next])<=k:
                            backProp(Gr.subgraph(subGs),ls,lt,k,next,ans)
                            backProp(G.subgraph(subGt),lt,ls,k,next,ans)
            qs=tqs
        else:
            tqt=[]
            for i in qt:
                for prev in Gr[i]:
                    if lt[prev]>lt[i]+1:
                        tqt.append(prev)
                        subGt.append(prev)
                        if (ls[prev]+lt[prev])>k and (lt[i]+1+ls[prev])<=k:
                            ans.append(prev)
                        lt[prev]=lt[i]+1
                        if (lt[prev]+ls[prev])<=k:
                            backProp(Gr.subgraph(subGs),ls,lt,k,prev,ans)
                            backProp(G.subgraph(subGt),lt,ls,k,prev,ans)
            qt=tqt
    return ls,lt

revised

只在两个未完成的圈内反向传播是有问题的

In [24]:
def bisearch_N(G,Gr,s,t,k,ans):
    def backProp_N(Gr,ls,lt,k,v,ans):
        for prev in Gr[v]:
            if lt[prev]>lt[v]+1 and ls[prev]+lt[v]+1<=k:
                if lt[prev]+ls[prev]>k:
                    ans.append(prev)
                lt[prev]=lt[v]+1
                backProp_N(Gr,ls,lt,k,prev,ans)
        
    ls=[k+1 for i in range(len(G))]
    lt=[k+1 for i in range(len(G))]
    ls[s]=0
    lt[t]=0
    qs=[s]
    qt=[t]

    for i in range(k):
        if len(qs)<len(qt) and len(qs)>0:
            tqs=[]
            for i in qs:
                for next in G[i]:
                    if ls[next]>ls[i]+1:
                        tqs.append(next)
                        if (ls[next]+lt[next])>k and (ls[i]+1+lt[next])<=k:
                            ans.append(next)
                        ls[next]=ls[i]+1
                        if(lt[next]+ls[next])<=k:
                            backProp_N(Gr,ls,lt,k,next,ans)
                            backProp_N(G,lt,ls,k,next,ans)
            qs=tqs
        else:
            tqt=[]
            for i in qt:
                for prev in Gr[i]:
                    if lt[prev]>lt[i]+1:
                        tqt.append(prev)
                        if (ls[prev]+lt[prev])>k and (lt[i]+1+ls[prev])<=k:
                            ans.append(prev)
                        lt[prev]=lt[i]+1
                        if (lt[prev]+ls[prev])<=k:
                            backProp_N(Gr,ls,lt,k,prev,ans)
                            backProp_N(G,lt,ls,k,prev,ans)
            qt=tqt
    return ls,lt

先产生两个子图再判定交集。则所有s-t k-hop路径都经过这一点集

直接找边??

In [71]:
def bisearch_S(G,Gr,s,t,k,ans):
    def backProp_S(Gr,ls,lt,k,v,ans):
        for prev in Gr[v]:
            if lt[prev]>lt[v]+1 and ls[prev]+lt[v]+1<=k:
                if lt[prev]+ls[prev]>k:
                    ans.append(prev)
                lt[prev]=lt[v]+1
                backProp_S(Gr,ls,lt,k,prev,ans)

    ls=[k+1 for i in range(len(G))]
    lt=[k+1 for i in range(len(G))]
    ls[s]=0
    lt[t]=0
    qs=[s]
    qt=[t]
    critical_node=[]

    for i in range(k):
        if len(qs)<len(qt):
            tqs=[]
            for i in qs:
                for next in G[i]:
                    if ls[next]>k:
                        tqs.append(next)
                        ls[next]=ls[i]+1
                        if (ls[next]+lt[next])<=k:
                            critical_node.append(next)
            qs=tqs
        else:
            tqt=[]
            for i in qt:
                for prev in Gr[i]:
                    if lt[prev]>k:
                        tqt.append(prev)
                        lt[prev]=lt[i]+1
                        if (lt[prev]+ls[prev])<=k:
                            critical_node.append(prev)
            qt=tqt
    for cn in critical_node:
        backProp(Gr,ls,lt,k,cn,ans)
        backProp(G,lt,ls,k,cn,ans)
    ans+=critical_node
    return ls,lt

In [ ]:
subG1=[]
bisearch(G,Gr,0,10,5,subG1)

In [60]:
subG2=[]
C_DFS(G,0,10,5,subG2)

2

In [25]:
subG3=[]
BC_DFS(G,Gr,0,10,5,subG3)

In [29]:
node1=subG
node2=list(set(i for j in subG2 for i in j))
node3=list(set(i for j in subG3 for i in j))

In [39]:
print(sorted(subG))
print(sorted(node2))
print(sorted(node3))

[0, 1, 3, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [42]:
ss

[0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]

In [43]:
st

[2, 1, 6, 1, 6, 1, 6, 1, 6, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

experiment

In [6]:
E=[]
with open("soc-LiveJournal1.txt","r") as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())

    for i in f.readlines():
        E.append((int(i.split()[0]),int(i.split()[1])))
        

# Directed graph (each unordered pair of nodes is saved once): soc-LiveJournal1.txt 

# Directed LiveJournal friednship social network

# Nodes: 4847571 Edges: 68993773

# FromNodeId	ToNodeId



In [8]:
np.save("./edges.npy",E)

In [11]:
socLiveJounal=nx.DiGraph()
socLiveJounal.add_edges_from(E)

In [12]:
np.save("./socLiveJournal.npy",socLiveJounal,allow_pickle=True)

In [13]:
socLiveJounalr=socLiveJounal.reverse()

In [14]:
np.save("./socLiveJournalr.npy",socLiveJounalr,allow_pickle=True)

In [64]:
start=time.time()
subG1=[]
BC_DFS(socLiveJounal,socLiveJounalr,0,1000,5,subG1)
end=time.time()
print(end-start)

1184.616861820221


In [17]:
start=time.time()
subG2=[]
bisearch(socLiveJounal,socLiveJounalr,0,1000,5,subG2)
end=time.time()
print(end-start)

140.4079475402832


In [20]:
len(sorted(subG2))

3470

In [21]:
len(sorted(subG1))

27658

In [22]:
start=time.time()
subG3=[]
C_DFS(socLiveJounal,0,1000,5,subG)
end=time.time()
print(end-start)

2538.1946880817413


In [26]:
start=time.time()
subG4=[]
bisearch_N(socLiveJounal,socLiveJounalr,0,1000,5,subG4)
end=time.time()
print(end-start)

3.6808390617370605


In [72]:
start=time.time()
subG5=[]
bisearch_S(socLiveJounal,socLiveJounalr,0,1000,5,subG5)
end=time.time()
print(end-start)

4.5010271072387695


In [75]:
len(set(subG5))

3573

In [74]:
len(subG5)

3573

In [44]:
len(set(subG4))

3496

In [31]:
set([j for j in i for i in subG1])

{'\t', '\n', '0', '4', '5', '7', '8'}

In [40]:
len(set([i[j] for j in range(2) for i in subG1]))

3435

In [43]:
len(set([i[j] for j in range(2) for i in subG3]))

3451

In [52]:
g4=set(subG4)
g3=set([i[j] for j in range(2) for i in subG3])
g1=set([i[j] for j in range(2) for i in subG1])

In [62]:
g=g3.difference(g1)
print(g)

{1215072, 244736, 362693, 1088070, 33447, 134281, 728873, 132939, 43403, 550029, 362701, 149074, 135315, 285301, 33370, 709467}


In [56]:
len(g1.difference(g3))

0

In [51]:
g1=set(subG1)